In [6]:
import meep as mp
import numpy as np
import matplotlib.pyplot as plt
import csv

def sourcepower_ldos(resolution):
    # === 설정 ===
    cell_size = mp.Vector3(3, 3, 0)
    fcen = 1.75
    df = 3.3
    nfreq = 300

    field_component = mp.Ex

    lambda_min = 0.4
    lambda_max = 0.7
    fmin = 1 / lambda_max
    fmax = 1 / lambda_min
    fcen_dft = 0.5 * (fmin + fmax)
    df_dft = fmax - fmin

    # 소스 설정
    source_center = mp.Vector3(0, 0, 0)
    sources = [
        mp.Source(
            src=mp.GaussianSource(frequency=fcen, fwidth=df),
            component=field_component,
            center=source_center,
            size=mp.Vector3(0, 0)
        )
    ]

    # PML 설정
    pml_layers = [mp.PML(0.3)]

    # 시뮬레이션 객체 생성
    sim = mp.Simulation(
        cell_size=cell_size,
        boundary_layers=pml_layers,
        default_material=mp.Medium(index=1.0),
        sources=sources,
        resolution=resolution
    )

    # === LDOS 객체 생성 ===
    freqs = np.linspace(fcen_dft - df_dft/2, fcen_dft + df_dft/2, nfreq)
    ldos_obj = mp._dft_ldos(freqs)

    # === 시뮬레이션 실행 ===
    sim.run(
        mp.dft_ldos(ldos=ldos_obj),
        until=50
    )

    # === 결과 얻기 ===
    ldos_power = np.array(sim.ldos_data)  # (W) 단위 power
    flux_freqs = np.array(mp.get_ldos_freqs(ldos_obj))  # (1/μm) 단위 주파수
    wavelengths = 1 / flux_freqs  # (μm) 단위 파장

    # === CSV 저장 ===
    csv_filename = f"ldos_sourcepower_res{resolution}.csv"
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Wavelength (μm)", "LDOS Power (a.u.)"])
        for wl, power in zip(wavelengths, ldos_power):
            writer.writerow([wl, power])

    print(f"CSV 파일 저장 완료: {csv_filename}")

    # === 플롯 저장 ===
    plt.figure(dpi=120)
    plt.plot(wavelengths, ldos_power, label="LDOS Power", color='blue')
    plt.xlabel("Wavelength (μm)")
    plt.ylabel("Power (a.u.)")
    plt.title(f"LDOS-based Source Power (Resolution={resolution})")
    plt.grid(True)
    plt.legend()
    
    plot_filename = f"ldos_sourcepower_res{resolution}.png"
    plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"플롯 이미지 저장 완료: {plot_filename}")

    return wavelengths, ldos_power



sourcepower_ldos(100)


-----------
Initializing structure...
time for choose_chunkdivision = 0.000178814 s
Working in 2D dimensions.
Computational cell is 3 x 3 x 0 with resolution 100
time for set_epsilon = 0.0493438 s
-----------
ldos0:, 1.4285714285714288, 0.7144502822053472
ldos0:, 1.4321548017200194, 0.7163367761240798
ldos0:, 1.43573817486861, 0.7182723184399172
ldos0:, 1.4393215480172004, 0.7201046747640735
ldos0:, 1.442904921165791, 0.7217945864538683
ldos0:, 1.4464882943143815, 0.7234556823990859
ldos0:, 1.4500716674629721, 0.7252303681313345
ldos0:, 1.4536550406115627, 0.7271371784203956
ldos0:, 1.457238413760153, 0.7290531525318662
ldos0:, 1.4608217869087436, 0.7308476855917039
ldos0:, 1.4644051600573342, 0.7325222170039039
ldos0:, 1.4679885332059248, 0.7342064168813847
ldos0:, 1.4715719063545154, 0.7360174183425182
ldos0:, 1.4751552795031058, 0.7379348486839823
ldos0:, 1.4787386526516964, 0.7398249259377658
ldos0:, 1.482322025800287, 0.7415853532299017
ldos0:, 1.4859053989488775, 0.74325414000442

(array([0.7       , 0.69824854, 0.69650582, 0.69477178, 0.69304636,
        0.69132948, 0.68962109, 0.68792112, 0.68622951, 0.6845462 ,
        0.68287113, 0.68120423, 0.67954545, 0.67789474, 0.67625202,
        0.67461724, 0.67299035, 0.67137129, 0.66976   , 0.66815642,
        0.66656051, 0.6649722 , 0.66339144, 0.66181818, 0.66025237,
        0.65869394, 0.65714286, 0.65559906, 0.6540625 , 0.65253313,
        0.65101089, 0.64949573, 0.64798762, 0.64648649, 0.6449923 ,
        0.643505  , 0.64202454, 0.64055088, 0.63908397, 0.63762376,
        0.63617021, 0.63472328, 0.6332829 , 0.63184906, 0.63042169,
        0.62900075, 0.62758621, 0.62617801, 0.62477612, 0.62338049,
        0.62199108, 0.62060786, 0.61923077, 0.61785978, 0.61649485,
        0.61513593, 0.61378299, 0.61243599, 0.61109489, 0.60975965,
        0.60843023, 0.6071066 , 0.60578871, 0.60447653, 0.60317003,
        0.60186916, 0.60057389, 0.59928418, 0.598     , 0.59672131,
        0.59544808, 0.59418027, 0.59291785, 0.59

In [11]:
import numpy as np
import matplotlib.pyplot as plt

# === 1. Meep 시뮬레이션 결과 ===
wavelengths_meep, ldos_power_meep = sourcepower_ldos(100)

# === 2. TXT 파일에서 Lumerical 데이터 불러오기 ===
# 파일명 지정 (네가 가지고 있는 txt 파일 이름으로 변경 가능)
txt_filename = 'ldos_source.txt'

# 파일 불러오기 (공백/탭/쉼표 등 구분자에 따라 수정 필요)
# 예시에서는 쉼표로 구분된다고 가정
data_txt = np.loadtxt(txt_filename, delimiter=',')

wavelengths_txt = data_txt[:, 0]  # 첫 번째 열: Wavelength (μm)
power_txt = data_txt[:, 1]        # 두 번째 열: Power (a.u.)

# === 3. 플롯 비교 ===
plt.figure(dpi=150)

# plt.plot(wavelengths_meep, ldos_power_meep, label='Meep LDOS', color='blue')
plt.plot(wavelengths_txt, power_txt, label='TXT File (e.g., Lumerical)', color='red', linestyle='--')

plt.xlabel("Wavelength (μm)")
plt.ylabel("Power (a.u.)")
plt.title("Meep vs TXT File - Source Power Comparison")
plt.grid(True)
plt.legend()
# plt.tight_layout()

# === 결과 저장 ===
comparison_plot_filename = "comparison_meep_vs_txt.png"
plt.savefig(comparison_plot_filename, dpi=300)
plt.close()

print(f"비교 플롯 저장 완료: {comparison_plot_filename}")


-----------
Initializing structure...
time for choose_chunkdivision = 0.000162125 s
Working in 2D dimensions.
Computational cell is 3 x 3 x 0 with resolution 100
time for set_epsilon = 0.048888 s
-----------
ldos0:, 1.4285714285714288, 0.7144502822053472
ldos0:, 1.4321548017200194, 0.7163367761240798
ldos0:, 1.43573817486861, 0.7182723184399172
ldos0:, 1.4393215480172004, 0.7201046747640735
ldos0:, 1.442904921165791, 0.7217945864538683
ldos0:, 1.4464882943143815, 0.7234556823990859
ldos0:, 1.4500716674629721, 0.7252303681313345
ldos0:, 1.4536550406115627, 0.7271371784203956
ldos0:, 1.457238413760153, 0.7290531525318662
ldos0:, 1.4608217869087436, 0.7308476855917039
ldos0:, 1.4644051600573342, 0.7325222170039039
ldos0:, 1.4679885332059248, 0.7342064168813847
ldos0:, 1.4715719063545154, 0.7360174183425182
ldos0:, 1.4751552795031058, 0.7379348486839823
ldos0:, 1.4787386526516964, 0.7398249259377658
ldos0:, 1.482322025800287, 0.7415853532299017
ldos0:, 1.4859053989488775, 0.743254140004423